## 도로시설물 관리현황 정보

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "도로시설물 관리현황 정보"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/도로시설물 관리현황 정보/selectIoFmFctStsList.csv'

In [2]:
# 개별 시설물 유지보수대장목록 수
g_totalCount = 0

In [3]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmFctStsList'

In [4]:
# REQPARAM = targetData["요청변수"].values[0]
# REQPARAM = REQPARAM.split(",")
# PRIMARYKEY = targetData["기본키"].values[0]
# REQPARAM

In [6]:
JSONKEY = "items"
DUMMY = 0

## 시설물 유지보수대장 목록 전용 스크랩 함수

In [15]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
                if(response.json()["response"]["body"].get(jsonkey) == None):                  
                    print(jsonkey+"is None !!!!")
                    break
                

                
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    jsondata = response.json()["response"]["body"][jsonkey]
                    if( jsondata == []):
                        print("items is empty")
                    else:
#                         jsondata["index"]=[0]
                        rowData = pd.DataFrame(jsondata)
                        print(rowData)
                        emptyPd = emptyPd.append(rowData)
#                         emptyPd = pd.merge(left = emptyPd , right = rowData, on = "fcno")
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [16]:

numOfRows = 1000
PAGEYN=0

In [17]:
class noDataFound(Exception): pass  # declare a label


In [23]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()

try:
    pageNo = 1
    global g_totalCount
    g_totalCount = 0

    while True:
        BASEPARAM={"serviceKey":APIKEY,"pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        if resultDf.empty: # 정상 데이터가 없는 경우
            print("NO DATA FOUND, CHECK CALS API")
            raise noDataFound()
        else:
            resultDfMerged = resultDfMerged.append(resultDf) 
            g_totalCount -= numOfRows
            print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
            if(g_totalCount <= 0):
                break
            pageNo += 1
except noDataFound:
    print("exception handled")
    
        

1 page scraping start
>totalCount 9274
       rn cmofNm       ofcNm     rutNm fctKdNm fctCfNm fctNm       ccwDt
0       1    서울청   수원국토관리사무소   일반국도3호선      교량      3종  진암3교  2002-12-01
1       2    서울청   수원국토관리사무소   일반국도3호선      교량      기타  진암2교  2002-12-01
2       3    서울청   수원국토관리사무소   일반국도3호선      교량      3종  진암1교  2002-12-01
3       4    서울청   수원국토관리사무소   일반국도3호선      교량      3종   진암교  2002-12-01
4       5    서울청   수원국토관리사무소   일반국도3호선      교량      3종   풍계교  1996-07-25
..    ...    ...         ...       ...     ...     ...   ...         ...
995   996    서울청  의정부국토관리사무소  일반국도77호선      교량      1종   오금교  1992-07-25
996   997    서울청  의정부국토관리사무소  일반국도87호선      교량      1종  영로대교  2017-01-01
997   998    원주청   홍천국토관리사무소   일반국도5호선      교량      3종   신림교  1995-03-19
998   999    원주청   홍천국토관리사무소   일반국도5호선      교량      기타   중간교  1987-12-31
999  1000    원주청   홍천국토관리사무소   일반국도5호선      교량      3종   치악교  1996-12-14

[1000 rows x 8 columns]
0 no pageNo
g_totalCount :  8274  numOfRows :  1000
1 page s

>totalCount 9274
       rn cmofNm      ofcNm     rutNm fctKdNm fctCfNm    fctNm       ccwDt
0    9001    부산청  진영국토관리사무소  일반국도14호선      교량      기타      남창교  1997-12-31
1    9002    부산청  진영국토관리사무소  일반국도14호선      교량      1종     남창2교  1997-12-31
2    9003    부산청  진영국토관리사무소  일반국도14호선      교량      3종     남창1교  1997-12-31
3    9004    부산청  진영국토관리사무소  일반국도14호선      교량      3종      운화교  1997-12-31
4    9005    부산청  진영국토관리사무소  일반국도14호선      교량      3종      진곡교  2006-12-20
..    ...    ...        ...       ...     ...     ...      ...         ...
269  9270    부산청  진영국토관리사무소  일반국도58호선      교량      기타      청학교  2018-09-15
270  9271    부산청  진영국토관리사무소  일반국도58호선      교량      기타      임천교  2018-09-15
271  9272    부산청  진영국토관리사무소  일반국도58호선      교량      기타      용성교  1988-08-01
272  9273    부산청  진영국토관리사무소  일반국도58호선      터널      2종  금곡터널(하)  2018-09-15
273  9274    부산청  진영국토관리사무소  일반국도79호선      교량      기타  죽남교(이관)  1980-01-01

[274 rows x 8 columns]
0 no pageNo
g_totalCount :  -726  numOfRows :  1000


In [24]:
resultDfMerged.shape

(9274, 8)

In [25]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

도로시설물 관리현황 정보 save compled


In [26]:
resultDfMerged.shape

(9274, 8)